In [2]:
import h5py
import os
import numpy as np
import zarr
import matplotlib.pyplot as plt
import mediapy as media
from PIL import Image
import cv2
from IPython.display import Video
folder = "/home/ravenhuang/gs-sim/data/successes"
file = os.listdir(folder)
data_paths = []
for f in file:
    data_path = os.path.join(folder, f)
    data_paths.append(data_path)

In [ ]:
for data in data_paths[:2]:
    fp = os.path.join(data, 'robot_data', 'robot_data_proprio.zarr')
    actions = zarr.load(fp)
    left, right = actions[:, :7], actions[:, 7:]
    obsp = os.path.join(data, 'camera_0', 'rgb')
    images = []
    for obsf in sorted(os.listdir(obsp), key=lambda x: int(x.split('.')[0])):
        obs_path = os.path.join(obsp, obsf)
        img = Image.open(obs_path)
        images.append(np.array(img))
    
    print(len(images))
    media.write_video(f"_obs.mp4",images, fps=30)
    Video("_obs.mp4")
    # out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (img.shape[0], img.shape[1]), isColor=False)

    # for frame in images:
    #     out.write(cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_GRAY2BGR))  # Convert to BGR for video compatibility

    # out.release()
    
    plt.figure()
    plt.plot(left[:,0], label='x')
    plt.plot(left[:,1], label='y')
    plt.plot(left[:,2], label='z')
    plt.legend()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import mediapy as media
data = datas[0]
proprio = data['state/cartesian/cartesian_pose']
action = data['action/cartesian_pose'][:,-7:]
left_arm_proprio = proprio[:, :7]
right_arm_proprio = proprio[:, 7:]
left_arm_action = action[:, :7]
right_arm_action = action[:, 7:]
right_arm_trans = right_arm_proprio[:, -3:]
# print(right_arm_trans)
print(np.mean(right_arm_trans, axis=0), np.std(right_arm_trans, axis=0))
print(right_arm_trans[:,1].max(0),right_arm_trans[:,1].min(0))
print(data['observation/camera/image/camera_rgb'].shape)
media.write_video(f"_obs.mp4",data['observation/camera/image/camera_rgb'], fps=30)


In [ ]:
from dp_gs.dataset.utils import quat_to_rot_6d, convert_multi_step_np, convert_delta_action, scale_action
data = datas[0]
proprio = data['state/cartesian/cartesian_pose']
action = data['action/cartesian_pose'][:,-7:]
left_arm_proprio = proprio[:, :7]
right_arm_proprio = proprio[:, 7:]
left_arm_action = action[:, :7]
right_arm_action = action[:, 7:]


def helper_load_action(f : h5py.File, start : int, end : int):
        # actions = f[self.action_key][start:end]
        actions = f[self.proprio_key][start+1:end+1]
        left, right = actions[:, :7], actions[:, 7:]
        # get gripper in a causal way
        left_g, right_g = f[self.joint_key][start + 1 : end + 1][:, -1][:, None], f[self.joint_key][start + 1 : end + 1][:, -2][:, None]

        left = np.concatenate([
            left[:, 4:],
            quat_to_rot_6d(left[:, :4]), 
            left_g * self.scale_left_gripper
        ], axis=1)
        right = np.concatenate([
            right[:, 4:],
            quat_to_rot_6d(right[:, :4]), 
            right_g * self.scale_right_gripper
        ], axis=1)
        return left, right

def helper_load_proprio(f : h5py.File, start : int, end : int):
    proprio = f[self.proprio_key][start:end]
    left, right = proprio[:, :7], proprio[:, 7:]
    left_g, right_g = f[self.joint_key][start:end][:, -1][:, None], f[self.joint_key][start:end][:, -2][:, None]
    left = np.concatenate([
        left[:, 4:],
        quat_to_rot_6d(left[:, :4]), 
        left_g * self.scale_left_gripper
    ], axis=1)
    right = np.concatenate([
        right[:, 4:],
        quat_to_rot_6d(right[:, :4]), 
        right_g * self.scale_right_gripper
    ], axis=1)
    return left, right
    

left_action = convert_delta_action(left_action, left_proprio)
right_action = convert_delta_action(right_action, right_proprio)

In [ ]:
print(right_arm_proprio[:,-3:])